In [ ]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import plotly.express as px
import scipy as sp
import osmnx as ox
import geopandas as gpd
import tensorflow as tf
import ruptures as rpt
import contextily as cx

import optuna
import torch
import pygsp

from matplotlib.ticker import ScalarFormatter, StrMethodFormatter, FormatStrFormatter, FuncFormatter
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from collections import Counter

from tslearn.metrics import dtw
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesScalerMinMax
from tslearn.shapelets import LearningShapelets, grabocka_params_to_shapelet_size_dict
from sklearn.metrics import mean_squared_error, confusion_matrix, auc
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sktime.transformations.panel.shapelet_transform import ShapeletTransform
from sktime.classification.shapelet_based import ShapeletTransformClassifier

from importlib import reload
from pyprojroot import here
ROOT_DIR = str(here())

import dario.models.mismatch_analysis as mma
from dario.models.maxdiv.maxdiv import maxdiv
mma = reload(mma)

insar_path = ROOT_DIR + "/data/raw/insar/"


plt.rcParams.update({'font.size': 20})
plt.rcParams.update({'font.family': 'Times New Roman'})

def compute_metric(df_test, cut=2, radius=15):

    df_metrics = []
    for cluster in sorted(df_test.cluster.unique()):

        df, nodes = mma.treat_nodes(df_test.query('cluster==@cluster'))
        G, nodes['subgraph'] = mma.NNGraph(nodes, radius=radius, subgraphs=True)

        df_metrics_cluster = []
        for sub_index in sorted(nodes.subgraph.unique())[1:]:

            subnodes = nodes.query('subgraph==@sub_index').copy()
            subdf = df[df.pid.isin(subnodes.pid)].copy()

            G = mma.NNGraph(subnodes, radius=radius)

            w, V = np.linalg.eigh(G.L.toarray())
            wh = np.ones(G.N)
            wh[w<cut] = 0
            Hh = V @ np.diag(wh) @ V.T

            smoothed = subdf[['pid', 'timestamp', 'smoothed' ]].pivot(index='pid', columns='timestamp')

            subdf['hf'] = np.abs((Hh @ smoothed.values).reshape((-1,), order='C'))

            df_metrics_cluster.append(subdf)

        df_metrics_cluster = pd.concat(df_metrics_cluster)
        df_metrics.append(df_metrics_cluster)

    df_metrics = pd.concat(df_metrics)
    return df_metrics

def detection(df_metrics, column_name='wse', threshold_min=1000, threshold_max=np.inf, selector='group',
              detection_param='detection_sum', detection_param_threshold=None):
    # df_relevant contains data from nodes that, at some point, have lower<=wse<=upper, and their neighbors.
    # nodes are put into groups if they are close to each other.

    if detection_param_threshold is None:
        detection_param_threshold = df_metrics.timestamp.nunique()//2

    df_relevant = mma.relevant_neighborhood(df_metrics, column_name=column_name,
                                            lower=threshold_min, upper=threshold_max,
                                            only_relevant=True, return_df=True, plot=False, filter_dates=False)

    # Treating disconnected nodes as individual groups. Assining new group values to these nodes
    new_group_values = df_relevant.query('group==0').pid.factorize()[0] + df_relevant.group.max()+1
    df_relevant.loc[df_relevant.group==0, 'group'] = new_group_values

    # Creates a detection column that is True for the timestamps when the metric is inside the thresholds
    # These are "partial detections". We will consider anomaly depending on how many partial detections are found
    df_relevant['detection'] = (df_relevant[column_name]>=threshold_min) & (df_relevant[column_name]<=threshold_max)

    # Counting partial detections (total, and consecutive occurances)
    df_detection = df_relevant.groupby('pid').agg({column_name:['max','mean'],
                                                    'detection':['sum',mma.consecutive_ones],
                                                    'group':'mean'}).reset_index()
    df_detection.columns = [f"{level1}_{level2}" if level2 else level1 for level1, level2 in df_detection.columns]
    df_detection.rename({'group_mean':'group'}, axis=1, inplace=True)


    # Querying for the cases where "actual detection" happens
    # Selected can return pids or groups, depending on the value of "selector"
    query = f'{detection_param}>{detection_param_threshold}'
    selected = df_detection.query(query)[selector].unique()

    return df_relevant, selected

def detectionv2(df_metrics, column_name='wse', quantile=0.995, detect_ratio = 0.5):

    
    detection_param_threshold = np.ceil(detect_ratio*df_metrics.timestamp.nunique())

    selected_pixels=[]

    for ts in df_metrics.timestamp.unique():
        df = df_metrics[df_metrics.timestamp==ts].copy()
        th = df_metrics[column_name].quantile(quantile)

        selected_pixels.append(df[df[column_name] >= th].pid.unique())

    flat_list = [item for sublist in selected_pixels for item in sublist]
    id_counts = Counter(flat_list)
    df_anomaly_count = pd.DataFrame({'pid':id_counts.keys(), 'count':id_counts.values()})

    faulty_pixels = df_anomaly_count.query('count>@detection_param_threshold').pid.unique()

    return faulty_pixels
    

def bounds_around_id(id, max_value, min_value=0, samples_before=30, samples_after=60):
    # Calculate the lower and upper bounds of the range
    # returns also:
    # case 0 if there are enough samples before and after
    # case 1 if not enough samples after
    # case 2 if not enough samples before    
    range_size= samples_before + samples_after
    lower_bound = id - samples_before
    upper_bound = id + samples_after

    case = 0
    
    # Adjust the range if it is too close to the edges
    if lower_bound < min_value:
        lower_bound = min_value
        upper_bound = min(range_size, max_value)
        case = 2
    elif upper_bound > max_value:
        upper_bound = max_value
        lower_bound = max(max_value - range_size, min_value)
        case = 1
    
    return lower_bound, upper_bound, case

def dtwkmeans(df, cluster_by='smoothed', n_clusters=6, n_init=5, savefile=None):

    data_ts = df[cluster_by].values.reshape((-1, df.pid.value_counts()[0],1))
    model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", n_init=n_init)
    y_pred = model.fit_predict(data_ts)

    # Plotting
    nrows = int(np.ceil(n_clusters/3))
    ncols = 3
    ratios = [3]
    ratios.extend([0.4,3]*(nrows-1))

    if n_clusters == 4:
        ncols = 2

    fig, ax = plt.subplots(nrows=2*nrows -1, ncols=ncols, figsize=(30,12), gridspec_kw={'height_ratios': ratios})

    for cluster in range(n_clusters):

        if df.timestamp.nunique()==data_ts.shape[1]:
            timestamp_list = df.timestamp.unique()
        else:
            timestamp_list = range(data_ts.shape[1])
            
        for timeseries in data_ts[y_pred == cluster]:
            ax[(cluster//ncols)*2,cluster%ncols].plot(timestamp_list, timeseries.ravel(), alpha=0.1)
        
        ax[(cluster//ncols)*2,cluster%ncols].text(0.05, 0.95, 
                                            str(sum(y_pred==cluster)),
                                            transform=ax[(cluster//ncols)*2,cluster%ncols].transAxes)

    for i in range(nrows):
        if ~i%2:
            ax[i,0].set_ylabel('Displacement')
            ax[i,0].set_ylabel('Displacement')
        else:
            for j in range(ncols):
                ax[i,j].set_visible(False)

    ax[-1,1].set_xlabel('Timestamp')
    if ncols==2:
        ax[-1,0].set_xlabel('Timestamp')

    fig.suptitle('TimeSeriesKmeans')
    fig.tight_layout()
    fig.subplots_adjust(hspace=0.05, wspace=0.05)
    if savefile is not None:
        plt.savefig(ROOT_DIR+savefile)
    plt.show()

    return y_pred

def plot_dtwkmeans(df, cluster_by='smoothed', savefile=None):

    data_ts = df[cluster_by].values.reshape((-1, df.pid.value_counts()[0],1))
    y_pred = df.drop_duplicates('pid')['grad_cluster'].values

    n_clusters = df.grad_cluster.nunique()

    # Plotting
    nrows = int(np.ceil(n_clusters/3))
    ncols = 3
    ratios = [3]
    ratios.extend([0.4,3]*(nrows-1))

    if n_clusters == 4:
        ncols = 2

    fig, ax = plt.subplots(nrows=2*nrows -1, ncols=ncols, figsize=(30,12), gridspec_kw={'height_ratios': ratios})

    for cluster in range(n_clusters):

        if df.timestamp.nunique()==data_ts.shape[1]:
            timestamp_list = df.timestamp.unique()
        else:
            timestamp_list = range(data_ts.shape[1])
            
        for timeseries in data_ts[y_pred == cluster]:
            ax[(cluster//ncols)*2,cluster%ncols].plot(timestamp_list, timeseries.ravel(), alpha=0.1)
        
        ax[(cluster//ncols)*2,cluster%ncols].text(0.05, 0.95, 
                                            str(sum(y_pred==cluster)),
                                            transform=ax[(cluster//ncols)*2,cluster%ncols].transAxes)

    for i in range(nrows):
        if ~i%2:
            ax[i,0].set_ylabel('Displacement')
            ax[i,0].set_ylabel('Displacement')
        else:
            for j in range(ncols):
                ax[i,j].set_visible(False)

    ax[-1,1].set_xlabel('Timestamp')
    if ncols==2:
        ax[-1,0].set_xlabel('Timestamp')

    fig.suptitle('TimeSeriesKmeans')
    fig.tight_layout()
    fig.subplots_adjust(hspace=0.05, wspace=0.05)
    if savefile is not None:
        plt.savefig(ROOT_DIR+savefile)
    plt.show()

### Preproc

In [ ]:
dataset = 'df_Porsgrunn_A1L2B'
df_orig = pd.read_parquet(ROOT_DIR+f"/data/interim/{dataset}.parq")
df_orig
print(df_orig.pid.nunique())

In [ ]:
mma.relevant_neighborhood(df_metrics, column_name='hf', lower=60, zoom=11, range_meters=15,
                          only_relevant=False, filter_dates=False, by_max=True)

### Testing new hf for IGARSS

In [ ]:
# Compute metrics
cut = 2 # frequency cut
radius = 20 # radius for constructing NNGraph

df = df_orig.copy()

df_metrics = compute_metric(df, cut, radius)

In [ ]:
df_orig.timestamp.nunique()

In [ ]:
selected = detectionv2(df_metrics, column_name='hf', quantile=0.995, detect_ratio=0.6)
len(selected)

In [ ]:
mma.plot_selected_map(df_metrics, column_name='hf', size='hf', selected_pixels=selected, only_relevant=False,
                     figsize=(1600,900))

### End of testing

In [ ]:
# ANOMALY FILTERING

# Doing two rounds of high-frequency filtering
# First round filters out the very-high frequency and avoids pixels with indirect high-frequency
# Second round filters out pixels that still have high-frequency that is not caused by the very-high frequency pixels

th1 = 70 # round 1 threshold
th2 = 40 # round 2 threshold
th_hits = 10 # number of timestamps hitting threshold to assign anomaly

cut = 2 # frequency cut
radius = 20 # radius for constructing NNGraph

df = df_orig.copy()

# round 1
df_metrics = compute_metric(df, cut, radius)
df_detection, selected = detection(df_metrics, column_name='hf', threshold_min=th1, selector='pid', 
                                   detection_param='detection_sum', detection_param_threshold=th_hits)
df = df[~df.pid.isin(selected)]

# # round 2
# df_metrics = compute_metric(df, cut, radius)
# df_detection, selected = detection(df_metrics, column_name='hf', threshold_min=th2, selector='pid',
#                                    detection_param='detection_sum', detection_param_threshold=th_hits)
# df = df[~df.pid.isin(selected)]

print(df.pid.nunique())

# Since metrics are not computed for lone pixels, merge into the original dataframe to include those
# Lone pixels get value 0 in the metric
df = df.merge(df_metrics[['hf']], how='left',left_index=True, right_index=True)
df = df.fillna(0)
# df.to_parquet(ROOT_DIR+f"/data/interim/{dataset}_filtered.parq")
df_filtered = df.copy()

In [ ]:
times = sorted(df.timestamp.unique())
midtime = len(times)//2
plt.rcParams.update({'font.size': 40})
plt.rcParams.update({'font.family': 'Times New Roman'})
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(26,12))
ax[0,0].hist(df_metrics.hf.values, log=True, density=True, bins=100)
ax[0,0].set_title('All data')

ax[0,1].hist(df_metrics.query('timestamp==@times[0]').hf.values, log=True, density=True, bins=100)
ax[0,1].set_title('First timestamp')

ax[1,0].hist(df_metrics.query('timestamp==@times[@midtime]').hf.values, log=True, density=True, bins=100)
ax[1,0].set_title('Central timestamp')

ax[1,1].hist(df_metrics.query('timestamp==@times[-1]').hf.values, log=True, density=True, bins=100)
ax[1,1].set_title('Last timestamp')

for axis in ax.flatten():
    axis.set_xlabel('Fault score')
    axis.tick_params(axis='both', which='both', labelsize=18)
    axis.grid()

plt.tight_layout()
fig.savefig(ROOT_DIR+"/models/outputs/figs/ReportESA/score_distribution.png", transparent=True)
plt.show()

In [ ]:
len(selected)

In [ ]:
df_metrics.query('timestamp==@times[-1]').hf.quantile(0.995)

In [ ]:
mma.plot_selected_map(df_metrics, column_name='hf', size='hf', selected_pixels=selected, figsize=(1600,900))

In [ ]:
np.random.seed(1)
df_test = df_orig[df_orig.pid.isin(selected)].drop_duplicates('pid')

fig, ax = plt.subplots(figsize=(16,9))
# colors = df_test.mean_velocity.astype('category').cat.codes
ax.scatter(df_test.longitude, df_test.latitude, s=1, cmap='Greys')

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
# Graph.plot(ax=ax, plot_name='')
cx.add_basemap(ax, crs='epsg:4326', source=cx.providers.OpenStreetMap.Mapnik)
plt.savefig(ROOT_DIR+"/models/outputs/figs/ReportESA/Porsgrunn_detections.png", transparent=True)

In [ ]:
df_single = df.copy()

In [ ]:
# ANOMALY FILTERING

# Doing two rounds of high-frequency filtering
# First round filters out the very-high frequency and avoids pixels with indirect high-frequency
# Second round filters out pixels that still have high-frequency that is not caused by the very-high frequency pixels

th1 = 70 # round 1 threshold
th2 = 40 # round 2 threshold
th_hits = 10 # number of timestamps hitting threshold to assign anomaly

cut = 2 # frequency cut
radius = 20 # radius for constructing NNGraph

df = df_orig.copy()

# round 1
df_metrics = compute_metric(df, cut, radius)
df_detection, selected = detection(df_metrics, column_name='hf', threshold_min=th1, selector='pid', 
                                   detection_param='detection_sum', detection_param_threshold=th_hits)
df = df[~df.pid.isin(selected)]

# round 2
df_metrics = compute_metric(df, cut, radius)
df_detection, selected = detection(df_metrics, column_name='hf', threshold_min=th2, selector='pid',
                                   detection_param='detection_sum', detection_param_threshold=th_hits)
df = df[~df.pid.isin(selected)]

print(df.pid.nunique())

# Since metrics are not computed for lone pixels, merge into the original dataframe to include those
# Lone pixels get value 0 in the metric
df = df.merge(df_metrics[['hf']], how='left',left_index=True, right_index=True)
df = df.fillna(0)
# df.to_parquet(ROOT_DIR+f"/data/interim/{dataset}_filtered.parq")
df_filtered = df.copy()

In [ ]:
df_double = df.copy()

In [ ]:
df.pid.nunique()

In [ ]:
detect_single = set(df_orig.pid.unique())-set(df_single.pid.unique())
detect_double = set(df_orig.pid.unique())-set(df_double.pid.unique())

In [ ]:
intersect = detect_single.intersection(detect_double)

In [ ]:
len(detect_single)

In [ ]:
len(detect_double)

In [ ]:
len(intersect)

In [ ]:
df_orig.pid.nunique()-df_single.pid.nunique()

In [ ]:
mma.relevant_neighborhood(df_metrics, column_name='hf', lower=30, zoom=11, range_meters=15,
                          only_relevant=False, filter_dates=False, by_max=True)

In [ ]:
df.to_parquet(ROOT_DIR+f"/data/interim/{dataset}_filtered.parq")

In [ ]:
df = pd.read_parquet(ROOT_DIR+f"/data/interim/{dataset}_filtered.parq")
df_filtered = df.copy()

In [ ]:
df = df_filtered.copy()
min_relevant_smoothed = 30
relevant_pids = (df[['pid','smoothed']]
                 .groupby('pid',as_index=False)
                 .agg(lambda x: np.ptp(x))
                 .query('smoothed>@min_relevant_smoothed')
                 .pid.unique()
)
print(len(relevant_pids))

df = df[df.pid.isin(relevant_pids)]

### KMEANS

In [ ]:
n_clusters = 6

data_ts = df.smoothed.values.reshape((-1, df.timestamp.nunique(),1))

model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", max_iter=20)
y_pred = model.fit_predict(data_ts)

df['tskmeans_6'] = np.repeat(y_pred, df.timestamp.nunique())
# df.to_parquet(ROOT_DIR+f"/data/interim/{dataset}_tskmeans.parq")

In [ ]:
df = pd.read_parquet(ROOT_DIR+f"/data/interim/{dataset}_tskmeans.parq")

In [ ]:
n_clusters = df.tskmeans_6.nunique()
data_ts = df.smoothed.values.reshape((-1, df.timestamp.nunique(),1))
y_pred = df.groupby('pid').tskmeans_6.min().values
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(30,12), gridspec_kw={'height_ratios': [3,0.4,3]})

for cluster in range(n_clusters):

    timestamp_list = df.timestamp.unique()
    for timeseries in data_ts[y_pred == cluster]:
        ax[(cluster//3)*2,cluster%3].plot(timestamp_list, timeseries.ravel(), alpha=0.1)
    
    ax[(cluster//3)*2,cluster%3].text(0.05, 0.95, 
                                        str(sum(y_pred==cluster)),
                                        transform=ax[(cluster//3)*2,cluster%3].transAxes)

ax[0,0].set_ylabel('Displacement')
ax[2,0].set_ylabel('Displacement')
ax[2,1].set_xlabel('Timestamp')
ax[1,0].set_visible(False)
ax[1,1].set_visible(False)
ax[1,2].set_visible(False)

fig.suptitle('TimeSeriesKmeans')

fig.tight_layout()
fig.subplots_adjust(hspace=0.05, wspace=0.05)
# plt.savefig(ROOT_DIR+'/models/outputs/clustering_TimeSeriesKmeans.png')
plt.show()

In [ ]:
n_clusters = 6

data_ts = df.query('tskmeans_6==0').smoothed.values.reshape((-1, df.timestamp.nunique(),1))

model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", max_iter=20)
y_pred = model.fit_predict(data_ts)

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(30,12), gridspec_kw={'height_ratios': [3,0.4,3]})

for cluster in range(n_clusters):

    timestamp_list = df.timestamp.unique()
    for timeseries in data_ts[y_pred == cluster]:
        ax[(cluster//3)*2,cluster%3].plot(timestamp_list, timeseries.ravel(), alpha=0.1)
    
    ax[(cluster//3)*2,cluster%3].text(0.05, 0.95, 
                                        str(sum(y_pred==cluster)),
                                        transform=ax[(cluster//3)*2,cluster%3].transAxes)

ax[0,0].set_ylabel('Displacement')
ax[2,0].set_ylabel('Displacement')
ax[2,1].set_xlabel('Timestamp')
ax[1,0].set_visible(False)
ax[1,1].set_visible(False)
ax[1,2].set_visible(False)

fig.suptitle('TimeSeriesKmeans')

fig.tight_layout()
fig.subplots_adjust(hspace=0.05, wspace=0.05)
# plt.savefig(ROOT_DIR+'/models/outputs/clustering_TimeSeriesKmeans.png')
plt.show()

### GRAD

In [ ]:
def center_on_gradmax(group):
    max_grad_abs_idx = group['grad_abs'].idxmax()  # Index of max 'grad_abs'
    max_smoothed = group.at[max_grad_abs_idx, 'smoothed']  # Value of 'smoothed' at max 'grad_abs'
    group['centered_on_gradmax'] -= max_smoothed  # Normalize 'smoothed'
    return group


def get_df_onset(df, threshold, clustering_length, prediction_range=30, pelt=None):
    # Receives a partial df for a single pixel

    if pelt is None:
        pelt = rpt.Pelt(model='rbf', min_size=60)

    ts = df.smoothed.values
    gs = df.grad_abs.values

    change = np.array(pelt.fit_predict(ts, pen=1))
    change_tuples = [(0, change[0])]+[(change[i]+1,change[i+1]) for i in range(len(change)-1)] # interval tuples
    
    id_max_gs = np.argmax(gs)
    id_max_pelt_segment = np.where(change>id_max_gs)[0][0]


    # If the max gradient is too early, the onset is probably not available
    if id_max_gs < prediction_range:
        onset=0
        case=0

    # If the onset happens at the beggining of the data. There is space before max_grad, but no prediction samples
    elif id_max_pelt_segment==0:
        onset = 0
        case = 1

    else:
        onset = change[id_max_pelt_segment-1]
        
        # check if previous interval has grad > threshold
        # For example, it goes down quickly before going up even more quickly. Pelt might separate two segments.
        prev_start = change_tuples[id_max_pelt_segment-1][0]
        prev_end = change_tuples[id_max_pelt_segment-1][1]
        if (gs[prev_start:prev_end]>threshold).any():
            onset = prev_start
        if onset == 0:
            case = 1
        else:
            # If there are enough samples for clustering after onset (2) or not (3)
            case = 2 if (onset+clustering_length)<=len(ts) else 3

    df_onset = df.iloc[onset:min(len(ts),onset+clustering_length)].copy()
    df_onset['onset'] = onset
    df_onset['onset_case'] = case
    return df_onset

In [ ]:
datasets = ['df_Trondheim_A1L2B', 'df_Kristiansand_A1L2B', 'df_Porsgrunn_A1L2B']
datasets = ['df_Kristiansand_A1L2B']
df_regions_all = []
for dataset in datasets:
    print(dataset)
    df = pd.read_parquet(ROOT_DIR+f"/data/interim/{dataset}_filtered.parq")

    df['grad'] = df.groupby('pid').smoothed.transform(np.gradient)
    df['grad2'] = df.groupby('pid').grad.transform(np.gradient)
    df['grad_abs'] = df.grad.abs()

    print('grads done')

    df['centered_on_gradmax'] = df['smoothed']
    df = df.groupby('pid', group_keys=False).apply(center_on_gradmax)
    df['grad_idmax'] = df.groupby('pid', as_index=False)['grad_abs'].transform(lambda x: np.argmax(x))

    print('centering done')

    threshold = 0.65

    # Group by 'pid' and find the row with the maximum 'grad_abs' for each sensor
    id_list = df.query('grad_abs>@threshold').pid.unique()
    df_list = df[df.pid.isin(id_list)]
    print(f'pixels with grad>threshold: {len(id_list)}')

    # max_grad_idx = df_list.groupby('pid', as_index=False)['grad_abs'].agg(lambda x: np.argmax(x))
    # # # Extract 91 timestamps around the maximum 'grad_abs' timestamp for each pixel
    # ntimestamps = df_list.timestamp.nunique()
    # df_regions = []
    # for pid, id in zip(max_grad_idx.pid.values, max_grad_idx.grad_abs.values):
    #     start, end, case = bounds_around_id(id, max_value=ntimestamps-1, samples_before=30, samples_after=60)
    #     df_temp = df_list.query('pid==@pid').iloc[start:end+1].copy()
    #     df_temp['grad_case'] = case
    #     df_regions.append(df_temp)
    # df_regions = pd.concat(df_regions)

    df_regions = []
    for pid in id_list:
        df_regions.append(get_df_onset(df.query('pid==@pid'), threshold=threshold, clustering_length=120))

    df_regions = pd.concat(df_regions)

    # df_regions['centered_on_gradmax'] = df_regions['smoothed']
    # def normalize_smoothed(group):
    #     max_grad_abs_idx = group['grad_abs'].idxmax()  # Index of max 'grad_abs'
    #     max_smoothed = group.at[max_grad_abs_idx, 'smoothed']  # Value of 'smoothed' at max 'grad_abs'
    #     group['centered_on_gradmax'] -= max_smoothed  # Normalize 'smoothed'
    #     return group

    # df_regions = df_regions.groupby('pid', group_keys=False).apply(normalize_smoothed)

    df_regions_all.append(df_regions)

df_regions_all = pd.concat(df_regions_all).reset_index()

display(df_regions_all.head())
display(df_regions_all.groupby('pid', as_index=False).onset_case.min().onset_case.value_counts())


In [ ]:
id = df.query('grad_abs>0.65').pid.unique()[4]
df_plot = df.query('pid==@id')

fig, ax = plt.subplots(figsize=(16,9))

ax.plot(df_plot.timestamp, df_plot.smoothed, label='Displacement')
ax.set_ylabel('Ground displacement [mm]')
ax.set_xlabel('Timestamp')
ax.legend(loc=2)
ax2 = ax.twinx()
ax2.plot(df_plot.timestamp, df_plot.grad, color='red')
ax2.hlines(y=-0.65, xmin=df_plot.timestamp.min(), xmax=df_plot.timestamp.max(), linestyles='dashdot', color='red')
ax2.set_ylabel('Gradient [mm/sample]')
plt.xlim([df_plot.timestamp.min(), df_plot.timestamp.max()])
ax.grid(axis='x')
ax2.legend(['Gradient','Threshold'], loc=0)

# plt.savefig(ROOT_DIR+f"/models/outputs/figs/ReportESA/gradient.png", transparent=True)
plt.show()

In [ ]:
# df_regions_all.to_parquet(ROOT_DIR+"/data/interim/df_regions_all.parq")

In [ ]:
df_regions = pd.read_parquet(ROOT_DIR+"/data/interim/df_regions_all.parq")

In [ ]:
df_regions.groupby('pid', as_index=False).grad_idmax.min().grad_idmax.hist(backend='plotly', nbins=500, cumulative=False)

In [ ]:
px.histogram(df_regions.groupby('pid', as_index=False).min(), x='onset', color='onset_case').show()

In [ ]:
n_clusters = 4
df_cluster = df_regions.query('onset_case==1 or onset_case==2').copy()
y_pred = dtwkmeans(df_cluster, cluster_by='centered_on_gradmax', n_clusters=n_clusters, n_init=5)
df_cluster['grad_cluster'] = np.repeat(y_pred, df_cluster.pid.value_counts()[0])

In [ ]:
df_cluster.to_parquet(ROOT_DIR+"/data/interim/df_cluster_all.parq")

In [ ]:
df_cluster.query('onset_case==2').onset.hist(backend='plotly')

In [ ]:
n_clusters = 4
df_cluster = df_regions.query('onset_case==1').copy()
y_pred = dtwkmeans(df_cluster, cluster_by='centered_on_gradmax', n_clusters=n_clusters, n_init=5)
df_cluster['grad_cluster'] = np.repeat(y_pred, df_cluster.pid.value_counts()[0])

In [ ]:
df_plot = df_list[df_list.pid.isin(df_cluster.pid.unique())]
df_plot = df_plot.merge(df_cluster.drop_duplicates('pid')[['pid','grad_cluster']], how='left', on='pid')
df_plot['idmax'] = df_plot.groupby('pid').grad_abs.transform(lambda x: np.argmax(x))
df_plot = df_plot.sort_values(['idmax','timestamp'])

for cluster in range(n_clusters):
    fig = px.line(df_plot.query('grad_cluster==@cluster'),
                x='timestamp', y='smoothed', animation_frame='pid', hover_data=['grad'])
    fig.update_yaxes(range=[-20, 20])  # Adjust the range as needed
    fig.show()

In [ ]:
for cluster in range(n_clusters):
    fig = px.line(df_cluster.groupby('pid', as_index=False)
                            .apply(lambda x: x.reset_index(drop=True)).reset_index()
                            .query('grad_cluster==@cluster'),
                x='level_1', y='centered_on_gradmax', animation_frame='pid', hover_data=['grad'])
    fig.update_yaxes(range=[-20, 20])  # Adjust the range as needed
    fig.show()

In [ ]:
import ruptures as rpt

plot_ids = [51, 56, 59, 71]

# for id in range(50,100):
for id in plot_ids:

    print(f'{id}-----------------------------------------------------------------------------------')
    ts_plot = df_list[df_list.pid==id_list[id]].smoothed.values
    ts_train = df_list[df_list.pid==id_list[id]].grad2.values

    # pelt = rpt.Pelt(model='rbf', min_size=60).fit(np.diff(ts_train))
    # pelt2 = rpt.Pelt(model='rbf', min_size=60).fit(np.diff(ts_plot))
    # result = pelt.predict(pen=0.5)
    # result2 = pelt2.predict(pen=1)

    pelt = rpt.Pelt(model='rbf', min_size=60)
    result = pelt.fit_predict(ts_plot, pen=1)

    # Plot the change points
    # rpt.display(ts_plot, result)
    rpt.display(ts_plot, result)
    plt.show()

In [ ]:
import ruptures as rpt

plot_ids = [51, 56, 59, 71]


count = 0
for id in plot_ids:
    count=count+1

    ts_plot = df_list[df_list.pid==id_list[id]].smoothed.values
    ts_train = df_list[df_list.pid==id_list[id]].grad2.values

    pelt = rpt.Pelt(model='rbf', min_size=60)
    result = pelt.fit_predict(ts_plot, pen=1)

    # Plot the change points
    # rpt.display(ts_plot, result)
    rpt.display(ts_plot, result, figsize=(16,4))
    plt.tick_params(
        axis='both',          # changes apply to the x-axis
        which='major',      # both major and minor ticks are affected
        left = False,
        labelleft = False,
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False) # labels along the bottom edge are off
    plt.box(False)
    plt.savefig(ROOT_DIR+f"/models/outputs/figs/ReportESA/pelt_{count}.png", transparent=True)

In [ ]:
retf.add_subplot() 

In [ ]:
f = fig[0]

In [ ]:
figu, ax = plt.subplots()
ax = f.get

In [ ]:
# DO ALSO HIGH FREQUENCY FOR GRAD

In [ ]:
df_cluster = pd.read_parquet(ROOT_DIR+"/data/interim/df_cluster_all.parq")

In [ ]:
df_cluster

In [ ]:
plot_dtwkmeans(df_cluster)

In [ ]:
n_clusters = 4
df_cluster_2 = df_cluster.query('grad_cluster==2 or grad_cluster==3').copy()
y_pred = dtwkmeans(df_cluster_2, cluster_by='centered_on_gradmax', n_clusters=n_clusters, n_init=5)
df_cluster_2['grad_cluster_2'] = np.repeat(y_pred, df_cluster.pid.value_counts()[0])

In [ ]:
df_plot = df[df.pid.isin(df_cluster_2.pid.unique())]
df_plot = df_plot.merge(df_cluster_2.drop_duplicates('pid')[['pid','grad_cluster_2']], how='left', on='pid')
df_plot['idmax'] = df_plot.groupby('pid').grad_abs.transform(lambda x: np.argmax(x))
df_plot = df_plot.sort_values(['idmax','timestamp'])

for cluster in range(n_clusters):
    fig = px.line(df_plot.query('grad_cluster_2==@cluster'),
                x='timestamp', y='smoothed', animation_frame='pid', hover_data=['grad'])
    fig.update_yaxes(range=[-20, 20])  # Adjust the range as needed
    fig.show()

### Shapelets

In [ ]:
datasets = ['df_Trondheim_A1L2B', 'df_Kristiansand_A1L2B', 'df_Porsgrunn_A1L2B']
df = []
for dataset in datasets:
    df.append(pd.read_parquet(ROOT_DIR+f"/data/interim/{dataset}_filtered.parq"))

df = pd.concat(df)

df['grad'] = df.groupby('pid').smoothed.transform(np.gradient)
df['grad2'] = df.groupby('pid').grad.transform(np.gradient)
df['grad_abs'] = df.grad.abs()


In [ ]:
df_cluster = pd.read_parquet(ROOT_DIR+"/data/interim/df_cluster_all.parq")

In [ ]:
plot_dtwkmeans(df_cluster)

In [ ]:
df_shape = df[df.pid.isin(df_cluster.query('onset_case==2').pid.unique())].copy()
df_shape = df_shape.merge(df_cluster[['pid','grad_cluster', 'onset']].drop_duplicates(), how='left', on='pid')

In [ ]:
def get_upto_onset(group):
    onset = group['onset'].min()  # Index of max 'grad_abs'
    group = group.iloc[onset-60:onset]
    return group

# dft = df_shape.groupby('pid', as_index=False).apply(get_upto_onset).reset_index(drop=True)

In [ ]:
df_shape = df[df.pid.isin(df_cluster.query('onset_case==2').pid.unique())].copy()
df_shape = df_shape.merge(df_cluster[['pid','grad_cluster', 'onset']].drop_duplicates(), how='left', on='pid')
df_shape = df_shape.groupby('pid', as_index=False).apply(get_upto_onset).reset_index(drop=True)

column_X = 'smoothed'
column_y = 'grad_cluster'
X = np.array(df_shape[column_X].values.reshape((-1, df_shape.pid.value_counts()[0])))
y = np.array(df_shape[column_y].values.reshape((-1, df_shape.pid.value_counts()[0]))[:,0].astype(int))

X = TimeSeriesScalerMinMax().fit_transform(X).squeeze()

In [ ]:
# Set the number of shapelets per size as done in the original paper
shapelet_sizes = grabocka_params_to_shapelet_size_dict(n_ts=n_ts,
                                                       ts_sz=ts_sz,
                                                       n_classes=n_classes,
                                                       l=0.1,
                                                       r=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
shp_clf = ShapeletTransformClassifier()
shp_clf.fit(X_train, y_train)

In [ ]:
y_pred = shp_clf.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred, normalize='true')

In [ ]:
df_cluster.groupby('pid', as_index=False).agg({'grad_abs':'max', 'grad_cluster':'min'}).groupby('grad_cluster').grad_abs.agg(['mean','std'])

### Detection

In [ ]:
datasets = ['df_Trondheim_A1L2B', 'df_Kristiansand_A1L2B', 'df_Porsgrunn_A1L2B']
df = []
for dataset in datasets:
    df.append(pd.read_parquet(ROOT_DIR+f"/data/interim/{dataset}_filtered.parq"))

df = pd.concat(df)

df['grad'] = df.groupby('pid').smoothed.transform(np.gradient)
df['grad2'] = df.groupby('pid').grad.transform(np.gradient)
df['grad_abs'] = df.grad.abs()

In [ ]:
healthy_list = df.groupby('pid',as_index=False).grad_abs.max().query('grad_abs<0.5').pid.unique()
healthy_selected = healthy_list[np.random.randint(0, high=len(healthy_list),size=1500)]

In [ ]:
df_healthy = df[df.pid.isin(healthy_selected)].copy()

In [ ]:
df_healthy_regions = []
for pid in healthy_selected:
    df_healthy_regions.append(get_df_onset(df_healthy.query('pid==@pid'), threshold=0.4, clustering_length=120))

df_healthy_regions = pd.concat(df_healthy_regions)

In [ ]:
df_healthy_regions.onset_case.value_counts()

In [ ]:
df_regions = pd.read_parquet(ROOT_DIR+"/data/interim/df_regions_all.parq")

In [ ]:
df_regions

In [ ]:
fig = px.line(df_test.groupby('pid', as_index=False).apply(lambda x: x.reset_index(drop=True)).reset_index(),
        x='level_1', y='zeromean', animation_frame='pid', hover_data=['grad'])
fig.update_yaxes(range=[-40, 40])  # Adjust the range as needed
fig.show()

In [ ]:
px.scatter(df.groupby('pid',as_index=False).grad.agg(['max','min']).reset_index(), x='max', y='min').show()

In [ ]:
df.groupby('pid', as_index=False)['grad'].agg(lambda x: x.iloc[np.argmax(np.abs(x))]).grad.hist(backend='plotly')

In [ ]:
id_list = (
            df
            .groupby('pid', as_index=False)['grad']
            .agg(lambda x: x.iloc[np.argmax(np.abs(x))])
            .query('grad>0.7 or grad<-0.7')
            .pid.unique()
)


In [ ]:
y_pred = dtwkmeans(df[df.pid.isin(id_list)])

In [ ]:
df_list = df[df.pid.isin(id_list)].copy()

In [ ]:
df_list['grad_cluster'] = np.repeat(y_pred, df_list.timestamp.nunique())


In [ ]:
px.line(df_list.query('grad_cluster==0'), x='timestamp', y='smoothed', animation_frame='pid').show()

In [ ]:
px.histogram(df, x='grad', facet_row='grad_cluster').show()

In [ ]:
df.groupby('pid').grad.apply(lambda x: x.abs().max()).hist(backend='plotly')

In [ ]:
for ind in range(0,50):
    preproc = ['normalize',
            'td',
            # 'deseasonalize',
            # 'detrend_linear'
            ]
    x = df_list[df_list.pid==df_list.pid.unique()[ind]].smoothed.values.reshape((1,-1))
    regions1 = maxdiv(x, useLibMaxDiv=True,  preproc=preproc, method='gaussian_cov',
                    extint_min_len=90, extint_max_len=90, num_intervals=1)
    
    regions2 = maxdiv(x, useLibMaxDiv=True,  preproc=preproc, method='parzen',
                extint_min_len=90, extint_max_len=90, num_intervals=1)


    fig = px.line(x.flatten())
    for region in regions1:
        fig.add_vline(x=region[0], line_color='green')
        fig.add_vline(x=region[1], line_color='green')

    for region in regions2:
        fig.add_vline(x=region[0], line_color='red')
        fig.add_vline(x=region[1], line_color='red')
    # fig.update_layout(title=f"{[r[2] for r in regions]}")
    fig.show()

In [ ]:
regions = maxdiv(df[df.pid==df.pid.unique()[0]].smoothed.values.reshape((1,-1)))

In [ ]:
regions

In [ ]:
df_test = df[['pid','smoothed','hf']].groupby('pid', as_index=False).max()

In [ ]:
plt.scatter(df_test.smoothed.values, df_test.hf.values)
plt.show()

In [ ]:
df.describe()

In [ ]:
mma = reload(mma)
mma.relevant_neighborhood(df.query('latitude>63.4355 and longitude<10.41'), column_name='hf', lower=0, upper=40, zoom=11,
                          only_relevant=False, filter_dates=False, by_max=True)

In [ ]:
df.query('latitude>63.4355 and longitude<10.41').iloc[430520]

In [ ]:
df_metrics.hf.describe()

In [ ]:
df_detection, selected = detection(df_metrics, column_name='hf', threshold_min=60, selector='pid',
                                   detection_param='detection_sum', detection_param_threshold=10)

In [ ]:
selected

In [ ]:
mma = reload(mma)
mma.relevant_neighborhood(df, column_name='hf', lower=0, zoom=11, range_meters=20,
                          only_relevant=False, filter_dates=False, by_max=True)